In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"
import pydeck as pdk
import mapboxgl
mapboxgl.__version__
from mapboxgl.viz import *
import os
from mapboxgl.utils import create_color_stops
from mapboxgl.utils import create_numeric_stops
import jenkspy

In [2]:
#원래 데이터
suwon_112 = pd.read_csv('1.수원시_112신고이력(격자_관할경계매핑).csv')
others_112 = pd.read_csv('3.3개시도_112신고이력(관할경계매핑).csv')
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_box_area = gpd.read_file('12.4개시도_치안안전센터_관할경계.geojson')
others_security_lamp = pd.read_csv('22.4개시도_보안등현황.csv')
others_security_bell = gpd.read_file('23.4개시도_안전비상벨현황.geojson')
others_cctv = pd.read_csv('24.4개시도_CCTV설치현황.csv')
suwon_building = pd.read_csv('26.수원시_건축물대장(표제부).csv')
others_park = pd.read_csv('27.4개시도_공원현황.csv')
others_police = pd.read_csv('39.4개시도_경찰관서현황.csv')
others_playground = pd.read_csv('40.4개시도_어린이놀이시설현황.csv')
others_land_use_plan = gpd.read_file('41.4개시도_토지이용계획도.geojson')
others_convenience = gpd.read_file('42.4개시도_편의점현황.geojson')



#구,동별로 되어 있는 데이터
suwon_homeless = pd.read_csv('13.수원시_노숙인자활시설현황.csv')
suwon_child_welfare = pd.read_csv('14.수원시_아동복지시설현황.csv')
suwon_child_center = pd.read_csv('15.수원시_지역아동센터시설현황.csv')
suwon_youth_study = pd.read_csv('16.수원시_청소년공부방현황.csv')
suwon_library = pd.read_csv('17.수원시_도서관현황.csv')
suwon_free_cafeteria =pd.read_csv('18.수원시_무료급식소현황.csv')
suwon_child_free_cafeteria = pd.read_csv('19.수원시_아동복지급식소현황.csv')
suwon_female_parcel = pd.read_csv('20.수원시_여성안심택배함현황.csv')
suwon_street_lamp = pd.read_csv('21.수원시_가로등현황.csv')


# 병원, 약국 데이터
병원_4개시 = pd.read_csv('4개시도_병원.csv')
약국_4개시 = pd.read_csv('4개시도_약국.csv')

### 치안센터 기준의 데이터프레임 생성

In [3]:
others_police_box_area.columns=['관할치안센터','관할경찰서','geometry']
others_police_box_area=others_police_box_area.drop_duplicates(['관할치안센터'])
others_police_box_area.reset_index(drop=True,inplace=True)

In [4]:
police_box_4sido = others_police_box_area.copy()
police_box_4sido = police_box_4sido[['관할치안센터','관할경찰서','geometry']]
police_box_4sido.sort_values(by=["관할치안센터"], inplace=True)
police_box_4sido.reset_index(inplace = True,drop=True)

### mapping & count 함수 생성

In [5]:
def point_police_box(data_df,oters_police_box_area,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby

# 카운트용 함수

def point_police_box_count(data_df,others_police_box,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"sum"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby    
def police_box_4sido_count(df,name):
    for i in range(len(df)):
        for j in range(len(police_box_4sido)):
            if df.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터']:
                    police_box_4sido.loc[j, name]=df.loc[i, 'cnt']

#### 지역명 매핑

In [6]:
area_list = []
for i in range(len(police_box_4sido)):
    area_list.append(police_box_4sido['관할경찰서'][i][0:2])
for i in range(len(area_list)):
    if (area_list[i]=='마산') | (area_list[i] == '진해'):
        area_list[i] = '창원'
police_box_4sido['지역'] = area_list


## 27.4개시도_공원현황 mapping

In [7]:
others_park_result_1, others_park_groupby=point_police_box(others_park,others_police_box_area,'park_gbn')
others_park_result=others_park_result_1.groupby(['관할치안센터', 'park_gbn']).count()
others_park_result.reset_index(inplace=True)
others_park_result=others_park_result[['관할치안센터','park_gbn','park_nm']]
police_box_4sido_count(others_park_groupby,'공원')

### 40.4개시도_어린이놀이시설현황/40.4개시도_어린이놀이시설현황_구별종류별

In [8]:
others_playground_result, others_playground_groupby=point_police_box(others_playground,others_police_box_area,'fac_loc')
police_box_4sido_count(others_playground_groupby,'놀이시설')

### 42.4개시도_편의점현황/42.4개시도_편의점현황_업체별수

In [9]:
others_convenience_result, others_convenience_groupby=point_police_box(others_convenience,others_police_box_area,'KSIC_NM')
others_convenience_result=others_convenience_result.groupby(['관할치안센터', 'KSIC_NM']).count()
others_convenience_result.reset_index(inplace=True)
others_convenience_result=others_convenience_result[['관할치안센터','KSIC_NM','BUSINESS_NM']]
police_box_4sido_count(others_convenience_groupby,'편의점')

## 22.4개시도_보안등현황 mapping

In [10]:
others_security_lamp_result, others_security_lamp_groupby=point_police_box_count(others_security_lamp,others_police_box_area,'securitylight_cnt')
police_box_4sido_count(others_security_lamp_groupby,'보안등')

## 23.4개시도_안전비상벨현황 mapping

In [11]:
others_security_bell_result = gpd.sjoin(others_police_box_area, others_security_bell, how='inner', op="intersects").drop(["index_right"],axis=1)
others_security_bell_groupby = others_security_bell_result.groupby("관할치안센터").agg({'bell_us':"count"}).reset_index().rename(columns={'bell_us':"cnt"})
others_security_bell_result=others_security_bell_result.groupby(['관할치안센터', 'bell_us'])['관할경찰서'].count()
others_security_bell_result=others_security_bell_result.to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})
police_box_4sido_count(others_security_bell_groupby,'비상벨')

## 24.4개시도_CCTV설치현황 mapping

In [12]:
others_cctv_result,others_cctv_groupby = point_police_box_count(others_cctv,others_police_box_area,'cctv_cnt')
others_cctv_result = others_cctv_result.groupby(['관할치안센터','purpose'])['cctv_cnt'].sum()
others_cctv_result=others_cctv_result.to_frame().reset_index()
police_box_4sido_count(others_cctv_groupby,'cctv')

In [13]:
others_cctv

,address,purpose,cctv_cnt,set_yr,lat,lon
0,경상남도 진주시 집현면 지내리 125,어린이보호,1,`2013-12,35.241319,128.093399
1,경상남도 진주시 집현면 지내리 125,어린이보호,4,`2013-12,35.242023,128.091990
2,경상남도 진주시 집현면 지내리 125,어린이보호,4,`2013-12,35.241209,128.092923
3,경상남도 진주시 진성면 상촌리 541-2,어린이보호,1,`2013-12,35.169310,128.224780
4,경상남도 진주시 진성면 상촌리 541-2,어린이보호,4,`2013-12,35.169223,128.225446
...,...,...,...,...,...,...
44974,서울특별시 성북구 길음동 635-7,어린이보호,4,`2010-12,37.613782,127.018663
44975,서울특별시 성북구 길음동 1280-4,어린이보호,4,`2010-12,37.610174,127.015332
44976,서울특별시 성북구 길음동 1168,어린이보호,4,`2010-12,37.609928,127.022334
44977,서울특별시 성북구 하월곡동 87-6,어린이보호,4,`2010-12,37.609267,127.032080


## 112 신고이력 관련 데이터 가공
- 20개 이상의 사건종별 데이터를 6개의 중분류로 매핑해주기
- 교통 / 질서유지 / 중요범죄 / 기타범죄 / 기타_경찰업무 / 타기관_기타

## 112 데이터 전처리 함수

In [14]:
def 사건종별_분류(x):
    교통 = ["교통사고","교통불편","교통위반"]
    질서유지 = ["기타경범","주취자","행패소란","보호조치","시비","위험방지","청소년비행","무전취식승차"]
    중요범죄 = ["절도","가정폭력","폭력"]
    기타범죄 = ["기타형사범","도박"]
    기타_경찰업무 = ["상담문의","분실습득","비상벨","실종(실종아동 등)"]
    타기관_기타 = ["기타_타기관","소음","서비스요청"]
    
    if x in 교통:
        return "교통"
    if x in 질서유지:
        return "질서유지"
    if x in 중요범죄:
        return "중요범죄"
    if x in 기타범죄:
        return "기타범죄"
    if x in 기타_경찰업무:
        return "기타_경찰업무"
    if x in 타기관_기타:
        return "타기관_기타"

In [15]:
def time_cut(x):
    if 6 > x >= 2:
        return '새벽'
    elif 11 > x >= 6:
        return '오전'
    elif 15 > x >= 11:
        return '점심'
    elif 18 > x >= 15:
        return '오후'
    elif 22 > x >= 18:
        return '저녁'
    else:
        return '심야'

In [16]:
others_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","관할치안센터","관할경찰서"]
suwon_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","gid","관할치안센터","관할경찰서"]

In [17]:
def 신고이력데이터_처리(data):
    
    others_112 = data.copy()
    others_112["접수완료시각"] = others_112["접수완료시각"].str.replace("'","")
    others_112["접수완료일자"] = others_112["접수완료일자"].astype(str)
    #others_112["도착일시"] = others_112["도착일시"].astype(int).astype(str)
    # 도착일시가 NaN인 경우 존재 -> 도착일시가 NaN이 아닌 경우에 대해서만 출동 소요시각 구하기

    others_112["접수완료일시"] = others_112["접수완료일자"] + others_112["접수완료시각"]
    others_112["접수완료일시"] = pd.to_datetime(others_112["접수완료일시"],format="%Y%m%d%H%M%S")

    others_112["년"] = others_112["접수완료일시"].apply(lambda x : x.year)
    others_112["월"] = others_112["접수완료일시"].apply(lambda x : x.month)
    others_112["일"] = others_112["접수완료일시"].apply(lambda x : x.day)
    others_112["요일"] = others_112["접수완료일시"].apply(lambda x : x.weekday())
    others_112["시"] = others_112["접수완료일시"].apply(lambda x : x.hour)
    others_112["분"] = others_112["접수완료일시"].apply(lambda x : x.minute)
    others_112["초"] = others_112["접수완료일시"].apply(lambda x : x.second)

    # 도착일시가 정확히 기록된 경우와 그렇지 않은 경우로 나누기

    def get_도착일시(x):
        if len(str(x)) < 14:
            return np.NaN
        else:
            return str(int(float(x)))

    others_112["도착일시"] = others_112["도착일시"].apply(lambda x : get_도착일시(x))
    others_112["도착일시"] = pd.to_datetime(others_112["도착일시"],format="%Y%m%d%H%M%S")

    # 출동하는데 걸린 소요 시간
    others_112["소요시간"] = others_112["도착일시"] - others_112["접수완료일시"]

    # 사건종별 중분류
    others_112["사건종별_중분류"] = others_112["사건종별"].apply(lambda x : 사건종별_분류(x))
    
    # 시간대별 분류
    others_112["시간대"] = others_112["시"].apply(lambda x : time_cut(x))
    
    return others_112

In [18]:
suwon_112_get = 신고이력데이터_처리(suwon_112)
others_112_get = 신고이력데이터_처리(others_112)

suwon_112_groupby = suwon_112_get.groupby("관할치안센터").agg({'gid':"count"}).reset_index().rename(columns={'gid':"cnt"})
suwon_112_get_groupby = suwon_112_get.groupby(['관할치안센터','사건종별_중분류'])['gid'].count().reset_index().rename(columns={'gid':"cnt"})

others_112_groupby = others_112_get.groupby("관할치안센터").agg({'사건종별':"count"}).reset_index().rename(columns={'사건종별':"cnt"})
others_112_get_groupby = others_112_get.groupby(['관할치안센터','사건종별_중분류'])['사건종별'].count().reset_index().rename(columns={'사건종별':"cnt"})

### 112 신고 데이터 치안센터 영역과 매핑하는 함수

In [19]:
def police_box_4sido_112_mapping(df1,df2):
    police_box_4sido_count(df1,'전체112')
    for i in range(len(df2)):
        for j in range(len(police_box_4sido)):
            if df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '기타_경찰업무':
                police_box_4sido.loc[j, '112_기타_경찰업무']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '중요범죄':
                police_box_4sido.loc[j, '112_중요범죄']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '교통':
                police_box_4sido.loc[j, '112_교통']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '타기관_기타':
                police_box_4sido.loc[j, '112_타기관_기타']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '질서유지':
                police_box_4sido.loc[j, '112_질서유지']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '기타범죄':
                police_box_4sido.loc[j, '112_기타범죄']=df2.loc[i, 'cnt']


In [20]:
police_box_4sido['전체112']=0
police_box_4sido['112_기타_경찰업무']=0
police_box_4sido['112_중요범죄']=0
police_box_4sido['112_교통']=0
police_box_4sido['112_타기관_기타']=0
police_box_4sido['112_질서유지']=0
police_box_4sido['112_기타범죄']=0

In [21]:
police_box_4sido_112_mapping(suwon_112_groupby,suwon_112_get_groupby)

In [22]:
police_box_4sido_112_mapping(others_112_groupby,others_112_get_groupby)

In [23]:
police_box_4sido['112_중요범죄_비율'] = round((police_box_4sido['전체112']/police_box_4sido['112_중요범죄'])*100)

### 8. 4개시도_유흥업소현황 mapping

In [24]:
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')[['BUSINESS_NM','COM_LC_NM','COM_MC_NM',
                'COM_SC_NM','KSIC_NM','SGG_NM',
       'ADB_EMD_NM','EMD_NM','ROAD_NM','geometry']]
others_alcohol.columns = ["상호명","상권업종대분류명","상권업종중분류명","상권업종소분류명","표준산업분류명",
                         "시군구명","행정동명","법정동명","도로명","geometry"]

In [25]:
others_alcohol_result = gpd.sjoin(others_police_box_area, others_alcohol, how='inner', op="intersects").drop(["index_right"],axis=1)
others_alcohol_result=others_alcohol_result.groupby(['관할치안센터'])['관할경찰서'].count().to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})
police_box_4sido_count(others_alcohol_result,'유흥업소')

In [26]:
# suwon_homeless = pd.read_csv('13.수원시_노숙인자활시설현황.csv')
# suwon_child_welfare = pd.read_csv('14.수원시_아동복지시설현황.csv')
# suwon_child_center = pd.read_csv('15.수원시_지역아동센터시설현황.csv')
# suwon_youth_study = pd.read_csv('16.수원시_청소년공부방현황.csv')
# suwon_library = pd.read_csv('17.수원시_도서관현황.csv')
# suwon_free_cafeteria =pd.read_csv('18.수원시_무료급식소현황.csv')
# suwon_child_free_cafeteria = pd.read_csv('19.수원시_아동복지급식소현황.csv')
# suwon_female_parcel = pd.read_csv('20.수원시_여성안심택배함현황.csv')
# suwon_street_lamp = pd.read_csv('21.수원시_가로등현황.csv')

### 약국 위치 매핑(보건)

In [27]:
others_pharmacy, others_pharmacy_groupby=point_police_box(약국_4개시,others_police_box_area,'기관주소')
police_box_4sido_count(others_pharmacy_groupby,'약국')

### 병원 위치 매핑(보건)

In [28]:
others_hospital, others_hospital_groupby=point_police_box(병원_4개시,others_police_box_area,'기관주소')
police_box_4sido_count(others_hospital_groupby,'병원')

### 지하철데이터 매핑

In [29]:
subway = pd.read_csv("전국도시철도역사정보표준데이터.csv",encoding="cp949")[["역사명","역사도로명주소","역경도","역위도"]]
subway.columns = ["기관명","기관주소","lon","lat"]
subway = subway[(subway["기관주소"].str.contains("서울")) | (subway["기관주소"].str.contains("수원"))]

In [30]:
others_subway_result, others_subway_groupby=point_police_box(subway,others_police_box_area,'기관주소')
police_box_4sido_count(others_subway_groupby,'지하철')

### 어린이집 데이터 매핑

In [31]:
어린이집_창원 = pd.read_excel("어린이집기본정보조회_창원.xls")
어린이집_수원 = pd.read_excel("어린이집기본정보조회_수원.xls")
어린이집_진주 = pd.read_excel("어린이집기본정보조회_진주.xls")
어린이집_서울 = pd.read_excel("어린이집기본정보조회_서울.xls")

others_baby = pd.concat([어린이집_창원,어린이집_수원,어린이집_진주,어린이집_서울])[["어린이집명","운영현황","주소","경도","위도"]]

others_baby = others_baby[(others_baby["운영현황"] == "정상") | (others_baby["운영현황"] == "재개")].reset_index(drop=True)
others_baby.columns = ["어린이집명","운영현황","주소","lon","lat"]
# 정상 운영되는 곳만

In [32]:
others_baby_result, others_baby_groupby=point_police_box(others_baby,others_police_box_area,'주소')
police_box_4sido_count(others_baby_groupby,'어린이집')

### 유치원 데이터 매핑

In [33]:
kindergarden = pd.read_csv('유치원 현황_위경도매핑.csv')

In [34]:
kindergarden_result, kindergarden_groupby=point_police_box(kindergarden,others_police_box_area,'교육청명')
police_box_4sido_count(kindergarden_groupby,'유치원')

### police_box_4sido 처리

In [35]:
police_box_4sido.fillna(0,inplace=True)
ar = ['공원', '놀이시설', '편의점', '보안등', '비상벨', 'cctv','전체112', '112_기타_경찰업무', '112_중요범죄', '112_교통', '112_타기관_기타', '112_질서유지','112_기타범죄', '유흥업소', '약국', '병원','지하철','어린이집']
for i in ar:
    police_box_4sido[i] = police_box_4sido[i].astype(int)

In [36]:
police_box_4sido.to_csv('치안센터_4개시.csv',encoding = 'utf-8')

# 사고다발지역 데이터 매핑
- 무단횡단
- 스쿨존
- 보행어린이
- 보행고령자
- 자전거

In [37]:
school_risk_area = pd.read_csv("스쿨존_사고다발.csv",encoding="cp949")
old_risk_area = pd.read_csv("고령_사고다발.csv",encoding="cp949")
bic_risk_area = pd.read_csv("자전거_사고다발.csv",encoding="cp949")
jay_risk_area = pd.read_csv("무단횡단_사고다발.csv",encoding="cp949")
child_risk_area = pd.read_csv("보행어린이_사고다발.csv",encoding="cp949")

In [38]:
school_risk_area["사고유형"] = "스쿨존내어린이사고"
old_risk_area["사고유형"] = "보행고령자사고"
bic_risk_area["사고유형"] = "자전거사고"
jay_risk_area["사고유형"] = "보행자무단횡단사고"
child_risk_area["사고유형"] = "보행어린이사고"

In [39]:
all_risk_area = pd.concat([school_risk_area,old_risk_area,bic_risk_area,jay_risk_area,child_risk_area],
                         axis=0).reset_index(drop=True)

geo_col이 문자열일 경우 polygon으로 변환하는 함수

In [40]:
def string_to_polygon(df,geo_col):
    data = df.copy()
    data["geometry"] = data[geo_col].apply(lambda x : Polygon(eval(x)["coordinates"][0]))
    data.drop(geo_col,axis=1,inplace=True)
    return data

In [41]:
all_risk_area_geo = string_to_polygon(all_risk_area,"다발지역폴리곤")
all_risk_area_geo = gpd.GeoDataFrame(all_risk_area_geo)

# 2020년 사고다발 데이터만 가져오기
all_risk_area_geo["사고다발지ID"] = all_risk_area_geo["사고다발지ID"].astype(str)
all_risk_area_geo = all_risk_area_geo[all_risk_area_geo["사고다발지ID"].str[:4] == "2020"]

In [42]:
all_risk_area_geo.to_file("사고유형별_사고다발지역.geojson",driver="GeoJSON")

In [43]:
all_risk_area_geo.reset_index(drop = True,inplace = True)

In [44]:
all_risk_area_geo_result = gpd.sjoin(others_police_box_area, all_risk_area_geo, how='inner', op="intersects").drop(["index_right"],axis=1)
all_risk_area_geo_result_groupby=all_risk_area_geo_result.groupby(['관할치안센터','사고유형'])['관할경찰서'].count().to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})


In [45]:
police_box_4sido['보행고령자사고'] = 0
police_box_4sido['자전거사고'] = 0
police_box_4sido['보행자무단횡단사고'] = 0
police_box_4sido['보행어린이사고'] = 0
police_box_4sido['스쿨존내어린이사고'] = 0

In [46]:
for i in range(len(all_risk_area_geo_result_groupby)):
    for j in range(len(police_box_4sido)):
        if all_risk_area_geo_result_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and all_risk_area_geo_result_groupby.loc[i, '사고유형'] == '보행고령자사고':
            police_box_4sido.loc[j, '보행고령자사고']=all_risk_area_geo_result_groupby.loc[i, 'cnt']
        elif all_risk_area_geo_result_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and all_risk_area_geo_result_groupby.loc[i, '사고유형'] == '자전거사고':
            police_box_4sido.loc[j, '자전거사고']=all_risk_area_geo_result_groupby.loc[i, 'cnt']
        elif all_risk_area_geo_result_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and all_risk_area_geo_result_groupby.loc[i, '사고유형'] == '보행자무단횡단사고':
            police_box_4sido.loc[j, '보행자무단횡단사고']=all_risk_area_geo_result_groupby.loc[i, 'cnt']
        elif all_risk_area_geo_result_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and all_risk_area_geo_result_groupby.loc[i, '사고유형'] == '보행어린이사고':
            police_box_4sido.loc[j, '보행어린이사고']=all_risk_area_geo_result_groupby.loc[i, 'cnt']
        elif all_risk_area_geo_result_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and all_risk_area_geo_result_groupby.loc[i, '사고유형'] == '스쿨존내어린이사고':
            police_box_4sido.loc[j, '스쿨존내어린이사고']=all_risk_area_geo_result_groupby.loc[i, 'cnt']
        

# 용도지역도 데이터 매핑

In [47]:
others_use_area = gpd.read_file('9.4개시도_용도지역도.geojson')
others_use_area.columns = ["관리번호","별칭","비고","고시일자","원천오브젝트ID","원천시군구코드","geometry"]

code = pd.read_csv("용도지역도_코드.csv")

In [48]:
others_use_area["코드"] = others_use_area["관리번호"].apply(lambda x : str(x)[20:26])
others_use_area_code = pd.merge(others_use_area,code,on="코드",how="left")

원천시군구코드
- 수원: 41110
- 창원: 48120
- 진주: 48170
- 서울: 원천시군구코드_서울

In [49]:
# 서울의 원천시군구코드 리스트 생성
원천시군구코드 = list(others_use_area['원천시군구코드'].unique())
for _ in range(3):
    원천시군구코드.pop()
원천시군구코드_서울 = 원천시군구코드

In [50]:
def 시군구별_용도지역도_격자별매핑(others_use_area,code,sgg_code,area,police_box_4sido):
    # others_use_area : 용도지역도 df
    # code : 용도지역도 코드 df
    # sgg_code : 시군구 코드 str
    # area : 상업,공업,주거,녹지 str
    # suwon_112_grid : 수원시 격자데이터
    
    others_use_area["코드"] = others_use_area["관리번호"].apply(lambda x : str(x)[20:26])
    others_use_area_code = pd.merge(others_use_area,code,on="코드",how="left")
    a = list(others_use_area_code[others_use_area_code["코드명"].isnull()].index)
    others_use_area_code.drop(a,inplace = True)
    # 용도지역도와 코드명 사이의 연결

    police_box_use_area = others_use_area_code[others_use_area_code["원천시군구코드"] == sgg_code]
    # 수원시 용도지역도

    police_box_use_area_sell = police_box_use_area[police_box_use_area["코드명"].str.contains(area)]
    # suwon_use_area_sell : 수원시의 00지역

    police_box_use_area_sell_intersect = gpd.overlay(police_box_4sido, police_box_use_area_sell, how="intersection")
    # suwon_use_area_sell_intersect : suwon_use_area_sell와 수원시의 격자와 겹치는 영역 산출

    police_box_use_area_sell_intersect = police_box_use_area_sell_intersect.to_crs(epsg="5186")
    police_box_use_area_sell_intersect["area"] = police_box_use_area_sell_intersect["geometry"].apply(lambda x :round(x.area,3))
    # Polygon 영역 넓이 구하기
    
    police_box_use_area_sell_intersect = police_box_use_area_sell_intersect.to_crs(epsg="4326")
    # 좌표계 반환

    police_box_use_area_sell_intersect_groupby = police_box_use_area_sell_intersect.groupby(["관할치안센터"]).agg({"area":"sum"}).reset_index()
    # gid 별 영역 넓이 합
    
    return police_box_use_area_sell_intersect_groupby.rename(columns={"area":area + "지역면적"})

In [51]:
for i in ['상업','공업','주거','녹지']:
    result = pd.DataFrame(index=range(0,1), columns=['관할치안센터'])
    for j in 원천시군구코드:
        a = 시군구별_용도지역도_격자별매핑(others_use_area,code,j,i,police_box_4sido)
        result = pd.concat([result,a])
    result.reset_index(inplace = True,drop = True)
    result.drop(0,inplace = True)
    globals()['서울_{}'.format(i)] = result
#서울_용도지역도
서울_상업 = 서울_상업.groupby(['관할치안센터']).agg({'상업지역면적':'sum'}).reset_index()
서울_공업 = 서울_공업.groupby(['관할치안센터']).agg({'공업지역면적':'sum'}).reset_index()
서울_주거 = 서울_주거.groupby(['관할치안센터']).agg({'주거지역면적':'sum'}).reset_index()
서울_녹지 = 서울_녹지.groupby(['관할치안센터']).agg({'녹지지역면적':'sum'}).reset_index()

In [52]:
#수원_용도지역도
수원_상업 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"41110","상업",police_box_4sido)
수원_공업 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"41110","공업",police_box_4sido)
수원_주거 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"41110","주거",police_box_4sido)
수원_녹지 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"41110","녹지",police_box_4sido)
        
# 창원_용도지역도
창원_상업 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48120","상업",police_box_4sido)
창원_공업 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48120","공업",police_box_4sido)
창원_주거 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48120","주거",police_box_4sido)
창원_녹지 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48120","녹지",police_box_4sido)

# 진주_용도지역도
진주_상업 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48170","상업",police_box_4sido)
진주_공업 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48170","공업",police_box_4sido)
진주_주거 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48170","주거",police_box_4sido)
진주_녹지 = 시군구별_용도지역도_격자별매핑(others_use_area,code,"48170","녹지",police_box_4sido)

치안센터_상업 = pd.concat([서울_상업,수원_상업,창원_상업,진주_상업]).reset_index().drop('index',axis=1)
치안센터_공업 = pd.concat([서울_공업,수원_공업,창원_공업,진주_공업]).reset_index().drop('index',axis=1)
치안센터_주거 = pd.concat([서울_주거,수원_주거,창원_주거,진주_주거]).reset_index().drop('index',axis=1)
치안센터_녹지 = pd.concat([서울_녹지,수원_녹지,창원_녹지,진주_녹지]).reset_index().drop('index',axis=1)

In [53]:
#치안센터_용도지역도 데이터 merge
result = police_box_4sido
#전체 면적 구하기
result = result.to_crs(epsg="5186")
result["전체면적"] = result["geometry"].apply(lambda x : round(x.area,3))
result = result.to_crs(epsg="4326")
전체면적 = result[['관할치안센터','전체면적']]


for area in ['상업','공업','주거','녹지']:
    df = globals()['치안센터_{}'.format(area)]
    for i in range(len(df)):
        for j in range(len(result)):
            if df.iloc[i,0] == result.iloc[j,0]:
                df.iloc[i,1] = round((df.iloc[i,1]/전체면적.loc[j,'전체면적'])*100)
                
for data in [치안센터_상업,치안센터_공업,치안센터_주거,치안센터_녹지]:
    police_box_4sido = pd.merge(police_box_4sido,data,on="관할치안센터",how="left").fillna(0)

# buffer 데이터 매핑

### 사고다발데이터 buffer 매핑

In [54]:
result_lst = []

buffer_intersect = gpd.overlay(police_box_4sido, all_risk_area_geo, how="intersection")
# suwon_112_grid(수원시의 grid)의 겹치는 영역 산출

buffer_intersect["area"] = buffer_intersect["geometry"].apply(lambda x : x.area)
# 넓이 구하기

for risk in all_risk_area_geo["사고유형"].value_counts().index:
    data = buffer_intersect[buffer_intersect["사고유형"] == risk]
    data_groupby = data.groupby(["관할치안센터"]).agg({"area":"sum"}).reset_index().rename(columns={"area":risk+'_buffer'})
    # 사고다발유형별 gid 별 영역 넓이 합

    result_lst.append(data_groupby)

for data in result_lst:
    police_box_4sido = pd.merge(police_box_4sido,data,on="관할치안센터",how="left").fillna(0)
police_box_4sido["사고다발지역_buffer"] = ( police_box_4sido["보행고령자사고_buffer"] + police_box_4sido["보행자무단횡단사고_buffer"] + police_box_4sido["자전거사고_buffer"] + 
                           police_box_4sido["보행어린이사고_buffer"] + police_box_4sido["스쿨존내어린이사고_buffer"] )
# result5 = result_buffer.drop("geometry",axis=1)

# 치안시설 buffer

In [55]:
def point_buffer_치안센터별매핑(data_df,police_box_4sido,col,buffer_num):
    # col : 경찰서, 치안센터, cctv (str)
    # buffer_num : 치안안전시설에 따라 10,300,1000 (int)
    
    df = data_df.copy()
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['Point'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='Point', crs = epsg4326).to_crs(epsg="5186")
    # Buffer 부여하기 위해서 Point 임시 epsg 5186 좌표계로 변환

    df["geometry"] = df["Point"].apply(lambda x : x.buffer(buffer_num))
    df = df.drop("Point",axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry').to_crs(epsg="4326")
    # Buffer 부여한 뒤에 epsg 4326 좌표계로 변환

    df['Point'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    # drop한 좌표 다시 넣어주기
    
    df_intersect = gpd.overlay(police_box_4sido,df, how="intersection")
    df_intersect = df_intersect.to_crs(epsg="5186")
    
    df_intersect[col+"_buffer"] = df_intersect["geometry"].apply(lambda x : round(x.area,3))
    # 면적은 5186 좌표계에서 구하기
    # 겹치는 면적 구하기 좌표계 바꿔서 구하자
    df_intersect = df_intersect.to_crs(epsg="4326")
    # 다시 좌표계 반환

    df_intersect_sum = df_intersect.groupby("관할치안센터").agg({col+"_buffer":"sum"}).reset_index()
    return df_intersect,df_intersect_sum

def point_buffer_치안센터별매핑_유흥업소(data_df,police_box_4sido,col,buffer_num):
    # col : 경찰서, 치안센터, cctv (str)
    # buffer_num : 치안안전시설에 따라 10,300,1000 (int)
    
    df = data_df.copy()
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['Point'] = df['geometry']
    df = gpd.GeoDataFrame(df, geometry='Point', crs = epsg4326).to_crs(epsg="5186")
    # Buffer 부여하기 위해서 Point 임시 epsg 5186 좌표계로 변환

    df["geometry_1"] = df["Point"].apply(lambda x : x.buffer(buffer_num))
    df = df.drop("Point",axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry_1').to_crs(epsg="4326")
    # Buffer 부여한 뒤에 epsg 4326 좌표계로 변환

    df['Point'] = df['geometry']
    # drop한 좌표 다시 넣어주기
    
    df_intersect = gpd.overlay(police_box_4sido,df, how="intersection")
    df_intersect = df_intersect.to_crs(epsg="5186")
    
    df_intersect[col+"_buffer"] = df_intersect["geometry"].apply(lambda x : round(x.area,3))
    # 면적은 5186 좌표계에서 구하기
    # 겹치는 면적 구하기 좌표계 바꿔서 구하자
    df_intersect = df_intersect.to_crs(epsg="4326")
    # 다시 좌표계 반환

    df_intersect_sum = df_intersect.groupby("관할치안센터").agg({col+"_buffer":"sum"}).reset_index()
    return df_intersect,df_intersect_sum

In [56]:
others_security_lamp_buffer,others_security_lamp_buffer_sum = point_buffer_치안센터별매핑(others_security_lamp,police_box_4sido,"보안등",10)
# 4개시 보안등

others_cctv_buffer,others_cctv_buffer_sum = point_buffer_치안센터별매핑(others_cctv,police_box_4sido,"CCTV",100)
# 4개시 CCTV

others_alcohol_buffer,others_alcohol_buffer_sum = point_buffer_치안센터별매핑_유흥업소(others_alcohol,police_box_4sido,"유흥업소",100)
# 4개시 유흥업소

In [57]:
# buffer면적/전체면적
for df in [others_security_lamp_buffer_sum,others_cctv_buffer_sum,others_alcohol_buffer_sum]:
    for i in range(len(df)):
        for j in range(len(전체면적)):
            if df.iloc[i,0] == 전체면적.iloc[j,0]:
                df.iloc[i,1] = round((df.iloc[i,1]/전체면적.loc[j,'전체면적'])*100)
# police_box_4sido에 buffer데이터 병합
for data in [others_security_lamp_buffer_sum,others_cctv_buffer_sum,others_alcohol_buffer_sum]:
    police_box_4sido = pd.merge(police_box_4sido,data,on="관할치안센터",how="left").fillna(0)

In [58]:
#경찰서 기준 데이터프레임 생성
police_4sido = police_box_4sido.groupby('관할경찰서').apply(lambda x: x.sum())
police_4sido=police_4sido.drop(['관할치안센터','관할경찰서','지역','112_중요범죄_비율'],axis=1).reset_index()

In [59]:
#경찰서 기준 데이터프레임에 지역 매핑
police_4sido['지역'] =police_4sido['관할경찰서'].str[:2]
police_4sido['지역']=police_4sido['지역'].replace('마산','창원')
police_4sido['지역']=police_4sido['지역'].replace('진해','창원')

In [60]:
#4개시_치안센터_final to_file
police_box_4sido.to_file("4개시_치안센터_final.geojson",driver="GeoJSON")

In [61]:
#4개시_경찰서_final geojson변환
others_police_area=others_police_area.rename(columns={'NAME':'관할경찰서'}).drop('PNAME',axis=1)
police_4sido = pd.merge(others_police_area,police_4sido, on ='관할경찰서' )

## 보안등 null값 처리 
- 평균값: 5522
- 하위 25%: 3927
- 2000이하인 값들은 이상치로 분류하여 해당지역의 평균값으로 대체

In [62]:
#보안등_이상치 처리
li = list(police_4sido[police_4sido['보안등'] < 2000].index)
police_4sido['보안등_이상치']=0
police_4sido.loc[li,'보안등_이상치'] = 1

In [63]:
police_4sido.loc[(police_4sido['보안등'] < 2000)&(police_4sido['지역']=='서울'), '보안등'] = round(police_4sido[(police_4sido['보안등']>=2000)&(police_4sido['지역']=='서울')]['보안등'].mean())

## cctv null값 처리 
- 평균값: 2398
- 하위 25%: 1376
- 500이하인 값들은 이상치로 분류하여 해당지역의 평균값으로 대체

In [64]:
#cctv_이상치 처리
li = list(police_4sido[police_4sido['cctv'] < 500].index)
police_4sido['cctv_이상치']=0
police_4sido.loc[li,'cctv_이상치'] = 1

In [65]:
police_4sido.loc[(police_4sido['cctv'] <500)&(police_4sido['지역']=='서울'), 'cctv'] = round(police_4sido[(police_4sido['cctv']>=500)&(police_4sido['지역']=='서울')]['cctv'].mean())

## 비상벨 null값 처리 
- 평균값: 455
- 하위 25%: 28
- 10이하인 값들은 이상치로 분류하여 해당지역의 평균값으로 대체

In [66]:
#비상벨_이상치 처리
li = list(police_4sido[police_4sido['비상벨'] < 10].index)
police_4sido['비상벨_이상치']=0
police_4sido.loc[li,'비상벨_이상치'] = 1

In [67]:
police_4sido.loc[(police_4sido['비상벨'] <10)&(police_4sido['지역']=='서울'), '비상벨'] = round(police_4sido[(police_4sido['비상벨']>=10)&(police_4sido['지역']=='서울')]['비상벨'].mean())

## 면적당 112 신고 데이터 비율 
- 112데이터 건수를 면적으로 나눈 후 1000000배 하여 조정

In [68]:
police_4sido = police_4sido.to_crs(epsg="5186")
police_4sido["area"] = police_4sido["geometry"].apply(lambda x :round(x.area,3))
# Polygon 영역 넓이 구하기

police_4sido = police_4sido.to_crs(epsg="4326")
# 좌표계 반환

In [69]:
# 112데이터/면적
li = ['전체112','112_기타_경찰업무','112_중요범죄','112_교통','112_타기관_기타','112_질서유지','112_기타범죄']
for i in li:
    police_4sido[i] = round((police_4sido[i]/police_4sido['area'])*1000000)
# 시설물데이터/면적
li = ['공원','놀이시설','편의점','보안등','비상벨','cctv','유흥업소','약국','병원','지하철','어린이집','유치원']
for i in li:
    police_4sido[i] = round((police_4sido[i]/police_4sido['area'])*100000000)

In [70]:
#4개시_경찰서_final to_file
police_4sido.to_file('4개시_경찰서_final.geojson',driver = 'GeoJSON')

# 체감 안전도 feature
- 사회적 무질서: 질서유지 관련 112 데이터(112_질서유지)
- 물리적 무질서: 시설물 관련 데이터(시설물)
- 교통안전: 교통관련 데이터(112_교통)
- 경찰활동: 전반적경찰노력도_점수 데이터(전반적경찰노력도_점수)

In [93]:
police_point = police_4sido.copy()

In [94]:
police_point.columns

Index(['관할경찰서', 'geometry', '공원', '놀이시설', '편의점', '보안등', '비상벨', 'cctv', '전체112',
       '112_기타_경찰업무', '112_중요범죄', '112_교통', '112_타기관_기타', '112_질서유지',
       '112_기타범죄', '유흥업소', '약국', '병원', '지하철', '어린이집', '유치원', '보행고령자사고',
       '자전거사고', '보행자무단횡단사고', '보행어린이사고', '스쿨존내어린이사고', '상업지역면적', '공업지역면적',
       '주거지역면적', '녹지지역면적', '보행고령자사고_buffer', '보행자무단횡단사고_buffer',
       '자전거사고_buffer', '보행어린이사고_buffer', '스쿨존내어린이사고_buffer', '사고다발지역_buffer',
       '보안등_buffer', 'CCTV_buffer', '유흥업소_buffer', '지역', '보안등_이상치', 'cctv_이상치',
       '비상벨_이상치', 'area'],
      dtype='object')

In [95]:
# 체감안전도 관련 feature들을 제외하고 drop
police_point.drop(list(police_point.columns)[21:39],axis=1,inplace=True)

In [96]:
#각 feature들의 점수화를 이용하기 위하여 Natural Breaks 사용
for i in list(police_point.columns)[2:21]:
    breaks=jenkspy.jenks_breaks(police_point[i],nb_class=10)
    police_point[i]=pd.cut(police_point[i],bins=breaks,
                               labels=[i for i in range(1,11)],
                               include_lowest=True)
    police_point[i] =police_point[i].astype(int) 

### 경찰활동 관련 Feature 생성

In [98]:
safe1=pd.read_csv("체감안전도(20180101).csv", header=None)
safe2=pd.read_csv("체감안전도(20180701).csv", header=None)
safe_effect1=pd.read_csv("체감안전도 영향요소 융합데이터(201980101).csv", header=None)
safe_effect2=pd.read_csv("체감안전도 영향요소 융합데이터(20180701).csv", header=None)

In [99]:
safe_header=['대상년월',
'경찰서',
'설문대상인원_전체',
'설문대상인원_1급지',
'설문대상인원_2급지',
'설문대상인원_3급지',
'설문대상인원_남성',
'설문대상인원_여성',
'설문대상인원_20대이하',
'설문대상인원_30대',
'설문대상인원_40대',
'설문대상인원_50대',
'설문대상인원_60대이하',
'종합체감안전도_점수',
'전반적안전도_점수',
'분야별안전도_점수',
'범죄안전도_점수',
'교통사고안전도_점수',
'법질서준수도_점수',
'절도폭력안전도_점수',
'강도살인안전도_점수',
'전반적경찰노력도_점수']

safe_effect_header=['대상년월',
'경찰서명',
'설문대상인원_전체',
'설문대상인원_1급지',
'설문대상인원_2급지',
'설문대상인원_3급지',
'설문대상인원_남성',
'설문대상인원_여성',
'설문대상인원_20대이하',
'설문대상인원_30대',
'설문대상인원_40대',
'설문대상인원_50대',
'설문대상인원_60대이하',
'종합체감안전도_점수',
'전반적안전도_점수',
'교통사고안전도_점수',
'법질서준수도_점수',
'절도폭력안전도_점수',
'강도살인안전도_점수',
'전반적경찰노력도_점수',
'월 평균 소득 금액',
'총 상주인구 수']


In [100]:
safe=pd.concat([safe1, safe2], axis=0).reset_index(drop=True)
safe.columns=safe_header

safe_effect=pd.concat([safe_effect1, safe_effect2], axis=0).reset_index(drop = True)
safe_effect.columns=safe_effect_header

In [101]:
safe=safe[['경찰서',
'종합체감안전도_점수',
'전반적안전도_점수',
'분야별안전도_점수',
'범죄안전도_점수',
'교통사고안전도_점수',
'법질서준수도_점수',
'절도폭력안전도_점수',
'강도살인안전도_점수',
'전반적경찰노력도_점수']]
safe_4 = safe[(safe["경찰서"].str[:2] == "서울") | 
            (safe["경찰서"].str[4:6] == "수원") | 
            (safe["경찰서"].str[2:4] == "창원") |
            (safe["경찰서"].str[2:4] == "마산") |
            (safe["경찰서"].str[2:4] == "진주")|
            (safe["경찰서"].str[2:4] == "진해") ]

In [102]:
safe_groupby=safe_4.groupby(['경찰서']).mean().reset_index()
safe_groupby.loc[0, '경찰서']="수원남부경찰서"
safe_groupby.loc[1, '경찰서']="수원서부경찰서"
safe_groupby.loc[2, '경찰서']="수원중부경찰서"
safe_groupby.loc[3, '경찰서']="마산동부경찰서"
safe_groupby.loc[4, '경찰서']="마산중부경찰서"
safe_groupby.loc[5, '경찰서']="진주경찰서"
safe_groupby.loc[6, '경찰서']="진해경찰서"
safe_groupby.loc[7, '경찰서']="창원서부경찰서"
safe_groupby.loc[8, '경찰서']="창원중부경찰서"
safe_groupby.sort_values(by="경찰서", inplace=True)
safe_groupby.reset_index(drop=True,inplace=True)
safe_groupby.rename(columns={'경찰서':'관할경찰서'},inplace = True)

In [103]:
#경찰활동 Feature 점수화
경찰활동 = safe_groupby[['관할경찰서','전반적경찰노력도_점수']]
경찰활동['전반적경찰노력도_점수'].describe()

breaks=jenkspy.jenks_breaks(경찰활동['전반적경찰노력도_점수'],nb_class=10)
경찰활동['전반적경찰노력도_점수']=pd.cut(경찰활동['전반적경찰노력도_점수'],bins=breaks,
                           labels=[i for i in range(1,11)],
                           include_lowest=True)
경찰활동['전반적경찰노력도_점수'] =경찰활동['전반적경찰노력도_점수'].astype(int) 

In [104]:
police_point = pd.merge(police_point,경찰활동,on = '관할경찰서')

물리적 무질서 
- 음의 시설물: '편의점','보안등','비상벨','cctv'
- 양의 시설물: '공원','놀이시설','유흥업소'
- 무질서가 높을수록 치안이 좋지 않다 

In [105]:
for i in ['편의점','보안등','비상벨','cctv','전반적경찰노력도_점수']:
    police_point[i]=police_point[i].replace([1,2,3,4,5,6,7,8,9,10],[10,9,8,7,6,5,4,3,2,1])

In [106]:
police_point['물리적_무질서'] = round(((police_point['편의점']+ police_point['보안등']+ police_point['비상벨']+ police_point['cctv']+ police_point['공원']+ police_point['놀이시설']+ police_point['유흥업소'] )/7),2 )
police_point['사회적_무질서'] = police_point['112_질서유지']
police_point['교통안전'] = police_point['112_교통']
police_point['경찰활동'] = police_point['전반적경찰노력도_점수']
police_point['체감안전도_지수'] = police_point['물리적_무질서']+police_point['사회적_무질서']+police_point['교통안전'] +police_point['경찰활동'] 

In [111]:
police_point.to_file('4개시도_체감안전도_지수.geojson',driver = 'GeoJSON')